In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam, SGD, Adagrad, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, Callback
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore', category=DeprecationWarning)

2023-05-08 23:12:55.485159: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 23:12:55.645487: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-08 23:12:55.648601: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-08 23:12:55.648622: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

#### Preprocessing

In [2]:
train, test = [], []
z_train, z_test = [], []
# chars =  ["bhA", "ba", "tA", "kaa", "tI"]
chars = ["hai", "ka", "rI", "sa", "pa"]

def read_ip(path):
    seq = []
    with open(path, 'r') as file:
        # Iterate over the lines in the file
        for line in file:
            data = [float(freq) for freq in line.split()]
            seq.append(data)
    return seq

for ch in chars:
    for f in os.listdir("Group24/CV_Data/" + str(ch)+"/Train"):
        train.append(read_ip("Group24/CV_Data/" + str(ch)+"/Train/" + str(f)))
        z_train.append(str(ch))
    for f in os.listdir("Group24/CV_Data/" + str(ch)+"/Test"):
        test.append(read_ip("Group24/CV_Data/" + str(ch) + "/Test/" + str(f)))
        z_test.append(str(ch))

In [3]:
train, val, z_train, z_val = train_test_split(train, z_train, test_size=0.2, random_state=42)

In [4]:
# label_map = {"bhA":0, "ba":1, "tA":2, "kaa":3, "tI":4}
label_map = {"hai":0, "ka":1, "rI":2, "sa":3, "pa":4}
z_train_onehot = tf.keras.utils.to_categorical([label_map[x] for x in z_train], 5)
z_val_onehot = tf.keras.utils.to_categorical([label_map[x] for x in z_val], 5)
z_test_onehot = tf.keras.utils.to_categorical([label_map[x] for x in z_test], 5)

In [5]:
train_tensor = tf.ragged.constant(train)
test_tensor = tf.ragged.constant(test)
val_tensor = tf.ragged.constant(val)

2023-05-08 23:13:00.469346: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-08 23:13:00.469369: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-08 23:13:00.469383: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (piyush-HP-Spectre-x360-Convertible-13-aw2xxx): /proc/driver/nvidia/version does not exist
2023-05-08 23:13:00.469620: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Model

In [6]:
model = Sequential()
model.add(SimpleRNN(256, batch_input_shape=(None, None, 39), return_sequences=True))
model.add(SimpleRNN(128, activation="relu", return_sequences=True))
model.add(SimpleRNN(64, activation="relu", return_sequences=True))
model.add(SimpleRNN(32, activation="relu", return_sequences=False))
model.add(Dense(64, activation="relu"))
model.add(Dense(5, activation="softmax"))
loss_fn = tf.keras.losses.CategoricalCrossentropy()

adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model.compile(loss="categorical_crossentropy", optimizer=adam_optimizer, metrics=['accuracy']) 
# Train the model
my_callbacks = [ EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=20),    TensorBoard(log_dir=f'./logdir/Q1/test')]
model_fit = model.fit(train_tensor, z_train_onehot, validation_data=(val_tensor, z_val_onehot), batch_size=len(train), epochs=10000, verbose=1, validation_split=0.0,callbacks=my_callbacks, shuffle=False, validation_batch_size=None)

hist_metric = 'accuracy'
print(f'epochs: {len(model_fit.history[hist_metric])}, acc: {model_fit.history[hist_metric][-1]}\n')
model.save(f'models/Q1/test.tf')

loss, accuracy = model.evaluate(test_tensor,z_test_onehot)

print("Test loss = ", loss)
print("Test accuracy = ", accuracy)

Epoch 1/10000
1/1 [==============================] - 4s 4s/step - loss: 2.1459 - accuracy: 0.2504 - val_loss: 1.5433 - val_accuracy: 0.3409
Epoch 2/10000
1/1 [==============================] - 1s 814ms/step - loss: 1.5726 - accuracy: 0.3385 - val_loss: 1.4614 - val_accuracy: 0.4062
Epoch 3/10000
1/1 [==============================] - 1s 859ms/step - loss: 1.4832 - accuracy: 0.3947 - val_loss: 1.3836 - val_accuracy: 0.4517
Epoch 4/10000
1/1 [==============================] - 1s 844ms/step - loss: 1.3981 - accuracy: 0.4403 - val_loss: 1.3106 - val_accuracy: 0.5000
Epoch 5/10000
1/1 [==============================] - 1s 861ms/step - loss: 1.3281 - accuracy: 0.4730 - val_loss: 1.2421 - val_accuracy: 0.5398
Epoch 6/10000
1/1 [==============================] - 1s 815ms/step - loss: 1.2628 - accuracy: 0.4929 - val_loss: 1.1758 - val_accuracy: 0.5767
Epoch 7/10000
1/1 [==============================] - 1s 767ms/step - loss: 1.1963 - accuracy: 0.5235 - val_loss: 1.1229 - val_accuracy: 0.5767
Ep